In [1]:
# !pip install transformers
!pip install datasets
! pip install -U accelerate
! pip install -U transformers
!pip install evaluate
!pip install sacrebleu
!pip install datasets transformers torch evaluate nltk rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 112.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:0

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import pandas as pd
df = pd.read_csv(r"/content/drive/MyDrive/PML/A1/data/training.csv")
df.columns

Index(['Unnamed: 0', 'toxic', 'neutral', 'toxicity score',
       'toxicity of neutral score'],
      dtype='object')

In [4]:
sub = df[df["toxicity score"] >=0.9]
sub = sub[sub["toxicity of neutral score"]<=0.2]
sub.shape

(357225, 5)

In [26]:
from datasets import load_dataset

data_files = {
    "train": r"/content/drive/MyDrive/PML/A1/data/training.csv",
    "test": r"/content/drive/MyDrive/PML/A1/data/testing.csv"
}
toxic_dataset = load_dataset("csv", data_files=data_files)

train_vald_dataset = toxic_dataset["train"].train_test_split(train_size=0.20, test_size=0.04, seed=20)
train_vald_dataset["validation"] = train_vald_dataset.pop("test")
train_vald_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'toxic', 'neutral', 'toxicity score', 'toxicity of neutral score'],
        num_rows: 92444
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'toxic', 'neutral', 'toxicity score', 'toxicity of neutral score'],
        num_rows: 18489
    })
})

In [27]:
train_vald_dataset["train"][2]

{'Unnamed: 0': 369772,
 'toxic': 'damn it, Miel, stop pulling that thing.',
 'neutral': 'Christ, Miel, stop picking away at that thing.',
 'toxicity score': 0.99936705827713,
 'toxicity of neutral score': 0.000439585361164}

In [28]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("rajistics/informal_formal_style_transfer")
input_sentence = train_vald_dataset["train"][2]["toxic"]
target = train_vald_dataset["train"][1]["neutral"]
output = tokenizer(input_sentence, text_target=target)
tokenizer.convert_ids_to_tokens(output["input_ids"])

['▁damn',
 '▁it',
 ',',
 '▁Mi',
 'e',
 'l',
 ',',
 '▁stop',
 '▁pulling',
 '▁that',
 '▁thing',
 '.',
 '</s>']

In [29]:
max_len = 128

def preprocessing_function(examples):
    inputs = examples["toxic"]
    targets = examples["neutral"]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=max_len)
    model_inputs["labels"] = tokenizer(targets, padding="max_length", truncation=True, max_length=max_len)["input_ids"]
    return model_inputs

tokenized_datasets = train_vald_dataset.map(
    preprocessing_function,
    batched=True,
    remove_columns=train_vald_dataset["train"].column_names
)

Map:   0%|          | 0/92444 [00:00<?, ? examples/s]

Map:   0%|          | 0/18489 [00:00<?, ? examples/s]

In [74]:
# tokenizer.pad_token = tokenizer.eos_token

# model.config.pad_token_id = tokenizer.pad_token_id

In [30]:
model = AutoModelForSeq2SeqLM.from_pretrained("rajistics/informal_formal_style_transfer")

# Assuming the T5 model has 12 layers in the encoder and 12 in the decoder
total_encoder_layers = len(model.encoder.block)
total_decoder_layers = len(model.decoder.block)

# Calculate 80% of the layers to freeze
num_encoder_layers_to_freeze = int(total_encoder_layers * 0.95)
num_decoder_layers_to_freeze = int(total_decoder_layers * 0.95)

# Freeze 80% of the encoder layers
for layer in model.encoder.block[:num_encoder_layers_to_freeze]:
    for param in layer.parameters():
        param.requires_grad = False

# Freeze 80% of the decoder layers
for layer in model.decoder.block[:num_decoder_layers_to_freeze]:
    for param in layer.parameters():
        param.requires_grad = False

# Optionally freeze embeddings
for param in model.shared.parameters():
    param.requires_grad = False


In [31]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
batch = data_collator([tokenized_datasets["train"][i] for i in range(10, 13)])
batch.keys()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


dict_keys(['input_ids', 'attention_mask', 'labels'])

In [32]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import tqdm
import torch

class StyleTransferAccuracy:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.tokenizer = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
        self.model = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

        # Move the model to the GPU
        self.model = self.model.to(self.device)

    def classify_preds(self, batch_size, preds):
        print('Calculating style of predictions')
        results = []

        for i in tqdm.tqdm(range(0, len(preds), batch_size)):
            batch = self.tokenizer(preds[i:i + batch_size], return_tensors='pt', padding=True)

            # Move the batch to the GPU
            batch = {key: value.to(self.device) for key, value in batch.items()}

            result = self.model(**batch)['logits'].argmax(1).float().data.tolist()
            results.extend([1 - item for item in result])

        return results

style_transfer_accuracy = StyleTransferAccuracy()
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    style_accuracy = style_transfer_accuracy.classify_preds(batch_size=32, preds=decoded_preds)

    # Calculate the average style accuracy
    average_style_accuracy = sum(style_accuracy) / len(style_accuracy)

    print(average_style_accuracy)
    # Return the metric as a dictionary
    return {"average_style_accuracy": average_style_accuracy}


Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments


args = Seq2SeqTrainingArguments(
    f"T5-detoxification",
    evaluation_strategy="steps",
    eval_steps=1000,
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    logging_steps=100,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [34]:
trainer.evaluate(max_length=max_len)

KeyboardInterrupt: ignored

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Average Style Accuracy
1000,0.055900,0.027967,0.253015
2000,0.048600,0.023900,0.252853
3000,0.045800,0.023840,0.252853
4000,0.047900,0.023849,0.252853
5000,0.046100,0.023859,0.252853
6000,0.045100,0.023864,0.252853


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Calculating style of predictions


100%|██████████| 578/578 [00:23<00:00, 24.17it/s]

0.25301530639839903


...

0.25285304775812645



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Calculating style of predictions


100%|██████████| 578/578 [00:23<00:00, 24.47it/s]

0.25285304775812645


In [20]:
model_path = r"/content/drive/MyDrive/University/PML/A1/gpt-2"
trainer.save_model(model_path)

In [21]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load the trained model and tokenizer
model_name_or_path = r"/content/drive/MyDrive/University/PML/A1/gpt-2"  # replace with your model's path
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)




T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [24]:
# Prepare the prompt
prompt = "What the fuck"

# Encode the prompt and move tensors to the same device as the model
inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=128)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate output sequence
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=128, num_return_sequences=1)

# Decode and print the output
detoxified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Detoxified text: {detoxified_text}")

Detoxified text: What the fuck
